In [1]:
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model
# import librosa
# import soundfile as sf
# import torch
# import pandas as pd

In [2]:
# import torch
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)

In [3]:
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
# model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self").to(device)
# model.config.output_hidden_states = True

In [4]:
# file_path = 'path_test.txt'
# with open(file_path, 'r') as file:
#     audio_paths = file.readlines()

In [5]:
# from tqdm import tqdm

# for audio_path in tqdm(audio_paths):
#     audio_path = audio_path.strip()

#     audio_input, sample_rate = sf.read(audio_path)

#     input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)

#     with torch.no_grad():
#         outputs = model(input_values)
        
# #     with torch.no_grad():
# #     outputs = model(**inputs)

#     # Get the last hidden state features
#     last_hidden_state_features = outputs.last_hidden_state.squeeze()

#     # Get the features from all layers
#     all_hidden_states = outputs.hidden_states

In [6]:
# import pandas as pd

# # Flatten the last hidden state features
# last_hidden_state_flat = last_hidden_state_features.view(-1, last_hidden_state_features.size(-1)).numpy()

# # Convert to DataFrame
# df_last_hidden_state = pd.DataFrame(last_hidden_state_flat)

# # Save the last hidden state features to a CSV file
# df_last_hidden_state.to_csv("last_hidden_state_features.csv", index=False)

# # Prepare DataFrames for all layers
# dfs_all_layers = []
# for i, layer_features in enumerate(all_hidden_states):
#     layer_flat = layer_features.view(-1, layer_features.size(-1)).numpy()
#     df_layer = pd.DataFrame(layer_flat)
#     dfs_all_layers.append(df_layer)

#     # Save each layer's features to a CSV file
#     df_layer.to_csv(f"layer_{i}_features.csv", index=False)

In [7]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 4  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'path_test.txt'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_test_features/layer_{target_layer}_features_test.ark'
output_scp = f'layer_test_features/layer_{target_layer}_features_test.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import torch
from tqdm import tqdm
import kaldiio

# Load the processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

# Specify the target layer number (0 for embeddings, 1 for first hidden layer, etc.)
target_layer = 5  # Adjust this according to which layer you want to extract features from

# Use CUDA if available
device = torch.device("cpu")
model.to(device)

# Read the list of audio file paths with speaker IDs
file_path = 'path_test.txt'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Lists to store speaker IDs and features for the target layer
speaker_ids = []
layer_features = []

# Process each audio file
for audio_file in tqdm(audio_files):
    audio_file = audio_file.strip()
   
    # Extract speaker ID from the file path
    speaker_id = audio_file.split('/')[-1].split('-')[0]  # adjust based on your file path format
    speaker_ids.append(speaker_id)
   
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_file)
   
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values.to(device)
   
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
   
    # Get the features from the target layer
    all_hidden_states = outputs.hidden_states
    layer_feature = all_hidden_states[target_layer].squeeze(0).cpu().numpy()  # Remove batch dimension and move to CPU
    layer_features.append(layer_feature)
   
    # Release GPU memory
    del input_values, outputs
    torch.cuda.empty_cache()

# Save the features in Kaldi format
output_ark = f'layer_test_features/layer_{target_layer}_features_test.ark'
output_scp = f'layer_test_features/layer_{target_layer}_features_test.scp'

with kaldiio.WriteHelper(f'ark,scp:{output_ark},{output_scp}') as writer:
    for j, feature in enumerate(layer_features):
        writer[speaker_ids[j]] = feature

print(f"Features from layer {target_layer} saved in Kaldi format.")
